In [ ]:
import os
import re
import dill
import requests
from bs4 import BeautifulSoup
from keras.utils import get_file

In [ ]:
def dump_dill(fname, obj):
    with open(fname, 'wb') as f:
        dill.dump(obj, f)
    return None

def load_dill(fname):
    with open(fname, 'rb') as f:
        return dill.load(f)

In [ ]:
keras_home = '''~/.keras/datasets'''

Find and download data

In [ ]:
base_url = '''https://dumps.wikimedia.org/'''
english_path = '''enwiki/'''
index = requests.get(base_url+english_path).text
soup_index = BeautifulSoup(index, 'html.parser')

In [ ]:
soup_index.find_all('a')

In [ ]:
available_dumps = [a['href'] for a in soup_index.find_all('a') if a.has_attr('href')]
available_dumps

Select a recent dump that is complete

In [ ]:
selected_dump = available_dumps[-3]

In [ ]:
dump_url = base_url + english_path + selected_dump

dump_html = requests.get(dump_url).text
soup_dump = BeautifulSoup(dump_html, 'html.parser')

Find urls to elements classified as files

In [ ]:
available_files = [li.a.get('href') for li in soup_dump.find_all('li', {'class': 'file'}) if 'xml' in li.a.get('href')]

In [ ]:
pattern = re.compile('.*pages-articles[0-9]+\.xml-p.*')
pattern

In [ ]:
target_files = [f[1:] for f in available_files if re.match(pattern, f)]
target_files

Download partitioned files to ~/.keras/datasets/enwiki/20200520/

In [ ]:
file_paths = []
for f in target_files:
    if not os.path.exists(keras_home+f):
        file_paths.append(get_file(f, base_url+f))

In [ ]:
file_paths

In [ ]:
dump_dill('../input/file_paths.dill', file_paths)